In [ ]:
TODO
- join info from dropped keywords
- json for each keyword

In [1]:
from pattern.en import pluralize, singularize
 
print pluralize('child') #children
print singularize('wolves') #wolf

children
wolf


In [2]:
from pattern.es import pluralize, singularize
 
print pluralize('calefactor') #children
print singularize('calefactores') #calefactor

calefactores
calefactor


In [5]:
print pluralize('ropas y calzados') #children
print singularize('ropas y calzados') #wolf

ropas y calzados
ropas y calzado


In [36]:
import unicodedata
def strip_accents(s):
    s= s.decode("utf-8")
    return ''.join((c for c in unicodedata.normalize('NFD', unicode(s)) if unicodedata.category(c) != 'Mn'))

strip_accents('Östblocket')

u'Ostblocket'

In [37]:
pluralize(strip_accents('grafía'))

u'grafias'

In [33]:
'grafía'.decode("utf-8") == u'grafía'

True

In [8]:
from pandas import DataFrame
import psycopg2

def connect_clm():
    
    host_url = 'group-clm-redshift.onap.io'
    db = "clm"

    conn = psycopg2.connect(
    host= host_url,
    user="johannes_braun",
    port=5439,
    password="b-A2#83e670d2d72",
    dbname=db)
 
    cur = conn.cursor() # create a cursor for executing queries
    return cur, conn

def execute_query(query):
    cur, conn = connect_clm()
    cur.execute(query);
    res = cur.fetchall()
    df = DataFrame(res, columns = [desc[0] for desc in cur.description])
    return df

In [175]:
top_100 = execute_query("""
select stats.*, cat.category_nk, cat.freq as cat_freq, lkp.category_name_lc, lkp.category_l1_nk, lkp.category_l1_name_lc
from recommendations.temp_johannes_search_stats stats
left join recommendations.recommend_search2category cat
on stats.search_term = cat.search_term
and cat.country_sk = 'letgo|latam|ar'
and cat.category_level = 2
and cat.rec_rank = 1
left join global_bi.dim_categories lkp
on lkp.country_sk = 'letgo|latam|ar'
and lkp.category_nk = cat.category_nk
order by distinct_sessions desc
""")

In [415]:
search_terms_with_cat = execute_query("""
with total_freq as (
 select country_sk, search_term, sum(freq) as total_freq
 FROM recommendations.recommend_search2category
 group by 1,2
),
full_breakdown as (
SELECT cat.*, tf.total_freq, freq :: DECIMAL(19,4)/total_freq :: DECIMAL(19,4) as significance, lkp.category_name_lc, lkp.category_l1_nk, lkp.category_l1_name_lc
FROM recommendations.recommend_search2category cat

inner join  total_freq tf on(
tf.country_sk = cat.country_sk
and tf.search_term = cat.search_term)

left join global_bi.dim_categories lkp
on lkp.country_sk = 'letgo|latam|ar'
and lkp.category_nk = cat.category_nk

where cat.country_sk = 'letgo|latam|ar'
--and cat.search_term = 'honda wave'
---and cat.category_level in (1,2,3)
--and cat.rec_rank in ( 1,2)
),

first_three_cats AS(
SELECT
country_sk,
search_term,
total_freq as total_cat_freq,

max(case when (category_level = 2 and rec_rank = 1) then category_nk else '0' end) as first_cat_id,

max(case when (category_level = 2 and rec_rank = 2) then category_nk else '0' end) as second_cat_id,
max(case when (category_level = 2 and rec_rank = 3) then category_nk else '0' end) as third_cat_id,

max(case when (category_level = 2 and rec_rank = 1) then category_name_lc else '' end) as first_cat_name,
max(case when (category_level = 2 and rec_rank = 2) then category_name_lc else '' end) as second_cat_name,
max(case when (category_level = 2 and rec_rank = 3) then category_name_lc else '' end) as third_cat_name,

max(case when category_level = 3 then significance else 0 end) :: DECIMAL(19,4) as no_cat_significance,
max(case when (category_level = 2 and rec_rank = 1) then significance else 0 end) :: DECIMAL(19,4) as first_cat_significance,
max(case when (category_level = 2 and rec_rank = 2) then significance else 0 end) :: DECIMAL(19,4) as second_cat_significance,
max(case when (category_level = 2 and rec_rank = 3) then significance else 0 end) :: DECIMAL(19,4) as third_cat_significance
from full_breakdown
where significance >0.05
and  category_level in (2,3)
-- and search_term in ('zapatillas', 'nike')
group by 1,2,3)

select 
stats.reply_conversion, 
stats.research_quote, 
stats.distinct_sessions, 
stats.num_subsequent_replies,
stats.reply_5min_conversion, 
stats.research_15sec_conversion, 
stats.rows,
cat.country_sk as cat_country_sk,
cat.total_cat_freq, 
cat.first_cat_id,
second_cat_id,
third_cat_id, 
first_cat_name,
second_cat_name,
third_cat_name,
no_cat_significance,
first_cat_significance, 
second_cat_significance,
third_cat_significance,
stats.search_term

from recommendations.temp_johannes_search_stats stats
left join first_three_cats cat
on stats.search_term = cat.search_term
and cat.country_sk = 'letgo|latam|ar'
order by distinct_sessions desc
""")

In [416]:
len(search_terms_with_cat)

82665

In [417]:
search_terms_with_cat[:10]

,reply_conversion,research_quote,distinct_sessions,num_subsequent_replies,reply_5min_conversion,research_15sec_conversion,rows,cat_country_sk,total_cat_freq,first_cat_id,second_cat_id,third_cat_id,first_cat_name,second_cat_name,third_cat_name,no_cat_significance,first_cat_significance,second_cat_significance,third_cat_significance,search_term
0,0.0532,0.0061,118822,19417,6317,729,118822,letgo|latam|ar,2195.0,362,0,0,Vehículos,,,0.2810,0.2756,0.0000,0.0000,fiat 147
1,0.0481,0.0096,114212,22293,5491,1097,114212,letgo|latam|ar,7510.0,362,800,0,Vehículos,Electrónica,,0.2194,0.1989,0.0617,0.0000,motos
2,0.0613,0.0105,101423,22720,6214,1067,101423,letgo|latam|ar,25689.0,815,888,853,Ropa y Accesorios de Mujer,Ropa y Accesorios de Hombre,Bebés y Niños,0.1946,0.1541,0.1271,0.0896,zapatillas
3,0.0505,0.0118,97578,17432,4930,1152,97579,letgo|latam|ar,4861.0,881,0,0,Deportes y Hobbies,,,0.2316,0.2135,0.0000,0.0000,bicicletas
4,0.0405,0.0145,97207,15176,3938,1409,97207,letgo|latam|ar,1321.0,362,0,0,Vehículos,,,0.2778,0.2694,0.0000,0.0000,fiat uno
5,0.0377,0.0153,88181,19924,3322,1347,88181,letgo|latam|ar,9600.0,362,853,0,Vehículos,Bebés y Niños,,0.2227,0.1928,0.0589,0.0000,autos
6,0.0530,0.0115,83509,16491,4425,958,83509,letgo|latam|ar,1571.0,362,0,0,Vehículos,,,0.2845,0.2762,0.0000,0.0000,gol
7,0.0819,0.0101,77226,28572,6322,780,77226,letgo|latam|ar,19082.0,800,0,0,Electrónica,,,0.2274,0.2133,0.0000,0.0000,celulares
8,0.0481,0.0074,75213,11209,3614,557,75213,letgo|latam|ar,1212.0,362,0,0,Vehículos,,,0.2920,0.2904,0.0000,0.0000,fiat 128
9,0.0506,0.0117,73770,11375,3736,865,73770,letgo|latam|ar,121.0,362,0,0,Vehículos,,,0.3140,0.3057,0.0000,0.0000,toyota hilux


In [418]:
search_terms_with_cat.columns

Index([u'reply_conversion', u'research_quote', u'distinct_sessions',
       u'num_subsequent_replies', u'reply_5min_conversion',
       u'research_15sec_conversion', u'rows', u'cat_country_sk',
       u'total_cat_freq', u'first_cat_id', u'second_cat_id', u'third_cat_id',
       u'first_cat_name', u'second_cat_name', u'third_cat_name',
       u'no_cat_significance', u'first_cat_significance',
       u'second_cat_significance', u'third_cat_significance', u'search_term'],
      dtype='object')

In [419]:
search_terms_with_cat['ascii'] = search_terms_with_cat['search_term'].apply(strip_accents)

In [420]:
search_terms_with_cat['singular'] = search_terms_with_cat['ascii'].apply(singularize)

In [421]:
'fiat 147' in search_terms_with_cat['search_term'].unique()

True

In [422]:
#u = search_terms_with_cat['search_term'].unique()
#search_terms_with_cat['is_valid'] = [ s['singular'] in u    for i, s in search_terms_with_cat.iterrows()]

In [423]:
#search_terms_with_cat['is_org'] = [ s['ascii']==s['singular'] for i, s in search_terms_with_cat.iterrows()]

In [424]:
search_terms_with_cat[search_terms_with_cat['search_term'] == 'peugeot chocado'].index.tolist()

[24400]

In [127]:
def safe_list_get (l, idx, default):
    try:
        return l[idx]
    except IndexError:
        return default

In [425]:
search_terms_with_cat['index'] = search_terms_with_cat.index

In [426]:
search_terms_with_cat[:5]

,reply_conversion,research_quote,distinct_sessions,num_subsequent_replies,reply_5min_conversion,research_15sec_conversion,rows,cat_country_sk,total_cat_freq,first_cat_id,...,second_cat_name,third_cat_name,no_cat_significance,first_cat_significance,second_cat_significance,third_cat_significance,search_term,ascii,singular,index
0,0.0532,0.0061,118822,19417,6317,729,118822,letgo|latam|ar,2195.0,362,...,,,0.2810,0.2756,0.0000,0.0000,fiat 147,fiat 147,fiat 147,0
1,0.0481,0.0096,114212,22293,5491,1097,114212,letgo|latam|ar,7510.0,362,...,Electrónica,,0.2194,0.1989,0.0617,0.0000,motos,motos,moto,1
2,0.0613,0.0105,101423,22720,6214,1067,101423,letgo|latam|ar,25689.0,815,...,Ropa y Accesorios de Hombre,Bebés y Niños,0.1946,0.1541,0.1271,0.0896,zapatillas,zapatillas,zapatilla,2
3,0.0505,0.0118,97578,17432,4930,1152,97579,letgo|latam|ar,4861.0,881,...,,,0.2316,0.2135,0.0000,0.0000,bicicletas,bicicletas,bicicleta,3
4,0.0405,0.0145,97207,15176,3938,1409,97207,letgo|latam|ar,1321.0,362,...,,,0.2778,0.2694,0.0000,0.0000,fiat uno,fiat uno,fiat uno,4


In [427]:
search_terms_with_cat['singular_pos'] = [ safe_list_get(search_terms_with_cat[search_terms_with_cat['ascii']==s['singular']].index.tolist(),0,False) for i, s in search_terms_with_cat.iterrows()]

In [428]:
# more popular plural version exists
a = list( search_terms_with_cat[(search_terms_with_cat['singular_pos']>search_terms_with_cat['index']) &(search_terms_with_cat['singular_pos'] >0) ]['singular_pos'])

In [429]:
# more popular singular version exists
b = list(search_terms_with_cat[(search_terms_with_cat['singular_pos']<search_terms_with_cat['index']) & (search_terms_with_cat['singular_pos'] >0) ]['index'])

In [430]:
bad_mask = a+b

In [431]:
bad_mask[:10]

[41, 543, 18, 208, 100, 70, 38, 974, 2503, 653]

In [432]:
len(bad_mask)

6293

In [433]:
search_terms_with_cat.loc[bad_mask[:10]]

,reply_conversion,research_quote,distinct_sessions,num_subsequent_replies,reply_5min_conversion,research_15sec_conversion,rows,cat_country_sk,total_cat_freq,first_cat_id,...,third_cat_name,no_cat_significance,first_cat_significance,second_cat_significance,third_cat_significance,search_term,ascii,singular,index,singular_pos
41,0.0589,0.0212,36897,8974,2175,782,36897,letgo|latam|ar,6394.0,362,...,,0.2219,0.1878,0.0619,0.0000,moto,moto,moto,41,41
543,0.0658,0.0124,7430,1827,489,92,7430,letgo|latam|ar,2903.0,815,...,Bebés y Niños,0.1904,0.1395,0.1181,0.0974,zapatilla,zapatilla,zapatilla,543,543
18,0.0654,0.0166,53392,12918,3493,885,53392,letgo|latam|ar,9360.0,881,...,,0.2292,0.2081,0.0000,0.0000,bicicleta,bicicleta,bicicleta,18,18
208,0.0425,0.0224,15652,4735,665,350,15652,letgo|latam|ar,3347.0,362,...,,0.2201,0.1819,0.0567,0.0000,auto,auto,auto,208,208
100,0.1032,0.0135,25163,11339,2596,339,25163,letgo|latam|ar,11728.0,800,...,,0.2268,0.2136,0.0000,0.0000,celular,celular,celular,100,100
70,0.0246,0.0096,31320,3150,770,301,31320,letgo|latam|ar,1064.0,362,...,,0.2612,0.2481,0.0000,0.0000,camioneta,camioneta,camioneta,70,70
38,0.0851,0.0092,37229,10627,3167,341,37230,letgo|latam|ar,7240.0,806,...,,0.2291,0.2147,0.0563,0.0000,heladera,heladera,heladera,38,38
974,0.0434,0.0280,4243,391,184,119,4243,letgo|latam|ar,49.0,815,...,Hogar y Jardín,0.2244,0.1224,0.1224,0.1020,masaje,masaje,masaje,974,974
2503,0.0640,0.0257,1673,359,107,43,1673,letgo|latam|ar,767.0,815,...,Ropa y Accesorios de Hombre,0.2346,0.2203,0.0586,0.0573,bota,bota,bota,2503,2503
653,0.0316,0.0232,6238,642,197,145,6238,letgo|latam|ar,172.0,362,...,,0.2732,0.2616,0.0000,0.0000,llanta,llanta,llanta,653,653


In [434]:
search_terms_with_cat[bad_rows][:10]

,reply_conversion,research_quote,distinct_sessions,num_subsequent_replies,reply_5min_conversion,research_15sec_conversion,rows,cat_country_sk,total_cat_freq,first_cat_id,...,third_cat_name,no_cat_significance,first_cat_significance,second_cat_significance,third_cat_significance,search_term,ascii,singular,index,singular_pos
18,0.0654,0.0166,53392,12918,3493,885,53392,letgo|latam|ar,9360.0,881,...,,0.2292,0.2081,0.0000,0.0000,bicicleta,bicicleta,bicicleta,18,18
38,0.0851,0.0092,37229,10627,3167,341,37230,letgo|latam|ar,7240.0,806,...,,0.2291,0.2147,0.0563,0.0000,heladera,heladera,heladera,38,38
41,0.0589,0.0212,36897,8974,2175,782,36897,letgo|latam|ar,6394.0,362,...,,0.2219,0.1878,0.0619,0.0000,moto,moto,moto,41,41
65,0.0822,0.0183,31914,11026,2624,584,31914,letgo|latam|ar,23049.0,815,...,Bebés y Niños,0.2135,0.1737,0.1124,0.0765,campera,campera,campera,65,65
70,0.0246,0.0096,31320,3150,770,301,31320,letgo|latam|ar,1064.0,362,...,,0.2612,0.2481,0.0000,0.0000,camioneta,camioneta,camioneta,70,70
84,0.0592,0.0079,28551,5772,1690,225,28551,letgo|latam|ar,2078.0,806,...,,0.2353,0.2170,0.0000,0.0000,cocinas,cocinas,cocina,84,75
100,0.1032,0.0135,25163,11339,2596,339,25163,letgo|latam|ar,11728.0,800,...,,0.2268,0.2136,0.0000,0.0000,celular,celular,celular,100,100
104,0.0178,0.0086,24726,2884,440,213,24726,letgo|latam|ar,978.0,362,...,,0.2443,0.1973,0.0000,0.0000,autos usados,autos usados,autos usado,104,72
106,0.0371,0.0077,24115,3105,894,185,24115,letgo|latam|ar,141.0,362,...,,0.2836,0.2836,0.0000,0.0000,ford ranger,ford ranger,ford ranger,106,106
114,0.0874,0.0225,23281,8584,2034,524,23281,letgo|latam|ar,1516.0,800,...,,0.2717,0.2658,0.0000,0.0000,iphone 6s,iphone 6s,iphone 6,114,74


In [435]:
bad_rows = search_terms_with_cat.index.isin(bad_mask)
clean_df = search_terms_with_cat[~bad_rows]

In [436]:
clean_df[:10]

,reply_conversion,research_quote,distinct_sessions,num_subsequent_replies,reply_5min_conversion,research_15sec_conversion,rows,cat_country_sk,total_cat_freq,first_cat_id,...,third_cat_name,no_cat_significance,first_cat_significance,second_cat_significance,third_cat_significance,search_term,ascii,singular,index,singular_pos
0,0.0532,0.0061,118822,19417,6317,729,118822,letgo|latam|ar,2195.0,362,...,,0.2810,0.2756,0.0000,0.0000,fiat 147,fiat 147,fiat 147,0,0
1,0.0481,0.0096,114212,22293,5491,1097,114212,letgo|latam|ar,7510.0,362,...,,0.2194,0.1989,0.0617,0.0000,motos,motos,moto,1,41
2,0.0613,0.0105,101423,22720,6214,1067,101423,letgo|latam|ar,25689.0,815,...,Bebés y Niños,0.1946,0.1541,0.1271,0.0896,zapatillas,zapatillas,zapatilla,2,543
3,0.0505,0.0118,97578,17432,4930,1152,97579,letgo|latam|ar,4861.0,881,...,,0.2316,0.2135,0.0000,0.0000,bicicletas,bicicletas,bicicleta,3,18
4,0.0405,0.0145,97207,15176,3938,1409,97207,letgo|latam|ar,1321.0,362,...,,0.2778,0.2694,0.0000,0.0000,fiat uno,fiat uno,fiat uno,4,4
5,0.0377,0.0153,88181,19924,3322,1347,88181,letgo|latam|ar,9600.0,362,...,,0.2227,0.1928,0.0589,0.0000,autos,autos,auto,5,208
6,0.0530,0.0115,83509,16491,4425,958,83509,letgo|latam|ar,1571.0,362,...,,0.2845,0.2762,0.0000,0.0000,gol,gol,gol,6,6
7,0.0819,0.0101,77226,28572,6322,780,77226,letgo|latam|ar,19082.0,800,...,,0.2274,0.2133,0.0000,0.0000,celulares,celulares,celular,7,100
8,0.0481,0.0074,75213,11209,3614,557,75213,letgo|latam|ar,1212.0,362,...,,0.2920,0.2904,0.0000,0.0000,fiat 128,fiat 128,fiat 128,8,8
9,0.0506,0.0117,73770,11375,3736,865,73770,letgo|latam|ar,121.0,362,...,,0.3140,0.3057,0.0000,0.0000,toyota hilux,toyota hilux,toyota hilux,9,9


In [437]:
def num_there(s):
    return any(i.isdigit() for i in s)

In [438]:
def ends_with_num_s(s):
    ews = False
    if len(s)>1:
        ews = (s[-2].isdigit()) & (s[-1]=="s")
    return ews

In [439]:
ends_with_num_s('iphons')

False

In [440]:
search_terms_with_cat['ends_with_number'] = search_terms_with_cat['search_term'].apply(ends_with_num_s)

In [441]:
iphone6s = search_terms_with_cat[search_terms_with_cat['ends_with_number']]

In [442]:
iphone6s[:10]

,reply_conversion,research_quote,distinct_sessions,num_subsequent_replies,reply_5min_conversion,research_15sec_conversion,rows,cat_country_sk,total_cat_freq,first_cat_id,...,no_cat_significance,first_cat_significance,second_cat_significance,third_cat_significance,search_term,ascii,singular,index,singular_pos,ends_with_number
114,0.0874,0.0225,23281,8584,2034,524,23281,letgo|latam|ar,1516.0,800,...,0.2717,0.2658,0.0000,0.0000,iphone 6s,iphone 6s,iphone 6,114,74,True
166,0.1032,0.0141,17765,7161,1834,250,17765,letgo|latam|ar,1888.0,800,...,0.2611,0.2537,0.0000,0.0000,iphone 5s,iphone 5s,iphone 5,166,307,True
1215,0.1062,0.0121,3465,1463,368,42,3465,letgo|latam|ar,301.0,800,...,0.2724,0.2558,0.0000,0.0000,iphone 4s,iphone 4s,iphone 4,1215,991,True
4947,0.0344,0.0652,813,105,28,53,813,letgo|latam|ar,15.0,362,...,0.2000,0.2000,0.2000,0.0000,renault 4s,renault 4s,renault 4,4947,445,True
8554,0.0584,0.0360,445,93,26,16,445,letgo|latam|ar,25.0,800,...,0.2800,0.2000,0.0800,0.0800,iphone 7s,iphone 7s,iphone 7,8554,136,True
8821,0.0837,0.0186,430,170,36,8,430,None,NaN,None,...,None,None,None,None,i phone 5s,i phone 5s,i phone 5,8821,11243,True
13710,0.1732,0.0197,254,183,44,5,254,letgo|latam|ar,54.0,800,...,0.2962,0.2962,0.0000,0.0000,6s,6s,6,13710,26726,True
15791,0.1395,0.0186,215,165,30,4,215,letgo|latam|ar,12.0,800,...,0.3333,0.3333,0.0000,0.0000,5s,5s,5,15791,29332,True
16426,0.0728,0.0485,206,57,15,10,206,letgo|latam|ar,13.0,800,...,0.2307,0.2307,0.1538,0.0000,4s,4s,4,16426,20238,True
16698,0.0746,0.0149,201,42,15,3,201,None,NaN,None,...,None,None,None,None,iphones 5s,iphones 5s,iphones 5,16698,21677,True


In [443]:
df = iphone6s.append(clean_df)

In [444]:
df = df.sort_values(by='distinct_sessions', ascending= False)

In [445]:
len(df)

76436

In [446]:
df[df['search_term'].str.startswith("iph")]['search_term'][:10]

16             iphone
74           iphone 6
114         iphone 6s
136          iphone 7
166         iphone 5s
166         iphone 5s
278     iphone 6 plus
326     iphone 7 plus
396    iphone 6s plus
714           iphones
Name: search_term, dtype: object

In [447]:
df[df['search_term'].str.startswith("ford r")]['search_term'][:10]

66                   ford rangers
889               ford ranger 4x4
929                 ford ranchero
2026                ford ranchera
3890    ford ranger cabina simple
4084              ford ranger xlt
4608          ford ranger limited
6206             ford ranger 2012
6577             ford ranger 2013
6800              ford ranger 4x2
Name: search_term, dtype: object

In [448]:
df.columns

Index([u'ascii', u'cat_country_sk', u'distinct_sessions', u'ends_with_number',
       u'first_cat_id', u'first_cat_name', u'first_cat_significance', u'index',
       u'no_cat_significance', u'num_subsequent_replies',
       u'reply_5min_conversion', u'reply_conversion',
       u'research_15sec_conversion', u'research_quote', u'rows',
       u'search_term', u'second_cat_id', u'second_cat_name',
       u'second_cat_significance', u'singular', u'singular_pos',
       u'third_cat_id', u'third_cat_name', u'third_cat_significance',
       u'total_cat_freq'],
      dtype='object')

In [449]:
df['popularity'] = df["distinct_sessions"]/100000

In [450]:
df["popularity"][df["popularity"]>1.0] = 1.0

/Users/johannes.braun/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [451]:
len(df)

76436

In [452]:
liquid_kws_df = df[df['reply_conversion']>0]

In [453]:
len(liquid_kws_df)

63676

In [454]:
liquid_kws_df[:100]

,ascii,cat_country_sk,distinct_sessions,ends_with_number,first_cat_id,first_cat_name,first_cat_significance,index,no_cat_significance,num_subsequent_replies,...,second_cat_id,second_cat_name,second_cat_significance,singular,singular_pos,third_cat_id,third_cat_name,third_cat_significance,total_cat_freq,popularity
0,fiat 147,letgo|latam|ar,118822,NaN,362,Vehículos,0.2756,0,0.2810,19417,...,0,,0.0000,fiat 147,0,0,,0.0000,2195.0,1.00000
1,motos,letgo|latam|ar,114212,NaN,362,Vehículos,0.1989,1,0.2194,22293,...,800,Electrónica,0.0617,moto,41,0,,0.0000,7510.0,1.00000
2,zapatillas,letgo|latam|ar,101423,NaN,815,Ropa y Accesorios de Mujer,0.1541,2,0.1946,22720,...,888,Ropa y Accesorios de Hombre,0.1271,zapatilla,543,853,Bebés y Niños,0.0896,25689.0,1.00000
3,bicicletas,letgo|latam|ar,97578,NaN,881,Deportes y Hobbies,0.2135,3,0.2316,17432,...,0,,0.0000,bicicleta,18,0,,0.0000,4861.0,0.97578
4,fiat uno,letgo|latam|ar,97207,NaN,362,Vehículos,0.2694,4,0.2778,15176,...,0,,0.0000,fiat uno,4,0,,0.0000,1321.0,0.97207
5,autos,letgo|latam|ar,88181,NaN,362,Vehículos,0.1928,5,0.2227,19924,...,853,Bebés y Niños,0.0589,auto,208,0,,0.0000,9600.0,0.88181
6,gol,letgo|latam|ar,83509,NaN,362,Vehículos,0.2762,6,0.2845,16491,...,0,,0.0000,gol,6,0,,0.0000,1571.0,0.83509
7,celulares,letgo|latam|ar,77226,NaN,800,Electrónica,0.2133,7,0.2274,28572,...,0,,0.0000,celular,100,0,,0.0000,19082.0,0.77226
8,fiat 128,letgo|latam|ar,75213,NaN,362,Vehículos,0.2904,8,0.2920,11209,...,0,,0.0000,fiat 128,8,0,,0.0000,1212.0,0.75213
9,toyota hilux,letgo|latam|ar,73770,NaN,362,Vehículos,0.3057,9,0.3140,11375,...,0,,0.0000,toyota hilux,9,0,,0.0000,121.0,0.73770


In [455]:
export = liquid_kws_df[['search_term', 'popularity', 'reply_conversion', 'research_quote', 'cat_country_sk', 
             'no_cat_significance', 'first_cat_significance', 'second_cat_significance', 'third_cat_significance',
            'first_cat_id', 'second_cat_id', 'third_cat_id','first_cat_name', 'second_cat_name', 'third_cat_name']]

In [456]:
export[:50]

,search_term,popularity,reply_conversion,research_quote,cat_country_sk,no_cat_significance,first_cat_significance,second_cat_significance,third_cat_significance,first_cat_id,second_cat_id,third_cat_id,first_cat_name,second_cat_name,third_cat_name
0,fiat 147,1.00000,0.0532,0.0061,letgo|latam|ar,0.2810,0.2756,0.0000,0.0000,362,0,0,Vehículos,,
1,motos,1.00000,0.0481,0.0096,letgo|latam|ar,0.2194,0.1989,0.0617,0.0000,362,800,0,Vehículos,Electrónica,
2,zapatillas,1.00000,0.0613,0.0105,letgo|latam|ar,0.1946,0.1541,0.1271,0.0896,815,888,853,Ropa y Accesorios de Mujer,Ropa y Accesorios de Hombre,Bebés y Niños
3,bicicletas,0.97578,0.0505,0.0118,letgo|latam|ar,0.2316,0.2135,0.0000,0.0000,881,0,0,Deportes y Hobbies,,
4,fiat uno,0.97207,0.0405,0.0145,letgo|latam|ar,0.2778,0.2694,0.0000,0.0000,362,0,0,Vehículos,,
5,autos,0.88181,0.0377,0.0153,letgo|latam|ar,0.2227,0.1928,0.0589,0.0000,362,853,0,Vehículos,Bebés y Niños,
6,gol,0.83509,0.0530,0.0115,letgo|latam|ar,0.2845,0.2762,0.0000,0.0000,362,0,0,Vehículos,,
7,celulares,0.77226,0.0819,0.0101,letgo|latam|ar,0.2274,0.2133,0.0000,0.0000,800,0,0,Electrónica,,
8,fiat 128,0.75213,0.0481,0.0074,letgo|latam|ar,0.2920,0.2904,0.0000,0.0000,362,0,0,Vehículos,,
9,toyota hilux,0.73770,0.0506,0.0117,letgo|latam|ar,0.3140,0.3057,0.0000,0.0000,362,0,0,Vehículos,,


In [463]:
synonyms_to_generate = search_terms_with_cat[bad_rows]
synonyms_to_generate = synonyms_to_generate[synonyms_to_generate['search_term']!=synonyms_to_generate['singular']]
synonyms_to_generate[['singular','search_term']]

/Users/johannes.braun/anaconda3/envs/python2/lib/python2.7/site-packages/pandas/core/ops.py:737: UnicodeWarning: Unicode unequal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  result = lib.vec_compare(x, y, op)


,singular,search_term
84,cocina,cocinas
104,autos usado,autos usados
114,iphone 6,iphone 6s
159,mesa,mesas
179,chocado,chocados
222,notebook,notebooks
244,casa,casas
260,tablet,tablets
311,guitarra,guitarras
319,caballo,caballos


In [468]:
export.columns

Index([u'search_term', u'popularity', u'reply_conversion', u'research_quote',
       u'cat_country_sk', u'no_cat_significance', u'first_cat_significance',
       u'second_cat_significance', u'third_cat_significance', u'first_cat_id',
       u'second_cat_id', u'third_cat_id', u'first_cat_name',
       u'second_cat_name', u'third_cat_name'],
      dtype='object')

In [470]:
export = export.rename( columns={"search_term": "query"})

In [471]:
len(export)

63676

In [481]:
type(export['popularity'][0])

numpy.float64

In [482]:
type(export['first_cat_significance'][0])

decimal.Decimal

In [484]:
export['first_cat_significance'] = export['first_cat_significance'].astype(float)

In [485]:
export['no_cat_significance'] = export['no_cat_significance'].astype(float)
export['reply_conversion'] = export['reply_conversion'].astype(float)
export['research_quote'] = export['research_quote'].astype(float)
export['second_cat_significance'] = export['second_cat_significance'].astype(float)
export['third_cat_significance'] = export['third_cat_significance'].astype(float)

In [508]:
import numpy as np
export['first_cat_id'] = export['first_cat_id'].replace('0', np.nan)
export['second_cat_id'] = export['second_cat_id'].replace('0', np.nan)
export['third_cat_id'] = export['third_cat_id'].replace('0', np.nan)

In [507]:
export['first_cat_significance'][export['first_cat_significance']<=0] = None
export['second_cat_significance'][export['second_cat_significance']<=0] = None
export['third_cat_significance'][export['third_cat_significance']<=0] = None

/Users/johannes.braun/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/johannes.braun/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/johannes.braun/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus

In [ ]:
export['third_cat_significance'][export['third_cat_significance']<=0] = None

In [472]:
export.to_csv("liquid_kws_ar_v2.csv", index=False)

In [513]:
sample_json = []
for i, s in export[0:50000].iterrows():
    d = s.dropna().to_dict()
    sample_json.append(d)

In [563]:
import json
with open('popular_queries.json', 'w') as fp:
    json.dump(sample_json, fp)

In [564]:
pwd

u'/Users/johannes.braun/dev/solr-6.3.0'

In [565]:
ls

CHANGES.txt                     example/
LICENSE.txt                     image_name.jpg
LUCENE_CHANGES.txt              licenses/
NOTICE.txt                      liquid_kws_ar_v2.csv
README.txt                      plural singular synonyms.ipynb
bin/                            popular_queries.json
contrib/                        server/
dist/                           twiggle python 2.ipynb
docs/


In [514]:
sample_json[:5]

[{'cat_country_sk': 'letgo|latam|ar',
  'first_cat_id': '362',
  'first_cat_name': 'Veh\xc3\xadculos',
  'first_cat_significance': 0.2756,
  'no_cat_significance': 0.281,
  'popularity': 1.0,
  'query': 'fiat 147',
  'reply_conversion': 0.0532,
  'research_quote': 0.0061,
  'second_cat_name': '',
  'third_cat_name': ''},
 {'cat_country_sk': 'letgo|latam|ar',
  'first_cat_id': '362',
  'first_cat_name': 'Veh\xc3\xadculos',
  'first_cat_significance': 0.1989,
  'no_cat_significance': 0.2194,
  'popularity': 1.0,
  'query': 'motos',
  'reply_conversion': 0.0481,
  'research_quote': 0.0096,
  'second_cat_id': '800',
  'second_cat_name': 'Electr\xc3\xb3nica',
  'second_cat_significance': 0.0617,
  'third_cat_name': ''},
 {'cat_country_sk': 'letgo|latam|ar',
  'first_cat_id': '815',
  'first_cat_name': 'Ropa y Accesorios de Mujer',
  'first_cat_significance': 0.1541,
  'no_cat_significance': 0.1946,
  'popularity': 1.0,
  'query': 'zapatillas',
  'reply_conversion': 0.0613,
  'research_quote

In [488]:
import requests
def index_solr_list_of_docs(sample_json):
    url = 'http://localhost:8983/solr/popular_queries/update?commit=true'
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    r = requests.post(url, headers = headers, 
                  data = json.dumps(sample_json)) 
    return r

In [592]:
import json
r = index_solr_list_of_docs(sample_json)

In [593]:
r.json()

{u'responseHeader': {u'QTime': 5312, u'status': 0}}

In [595]:
import requests
def suggest(search_term):
    #url = 'http://localhost:8983/solr/solrpedia_instant/select?q=query_ngram:'+search_term+'&%20sort=popularity%20desc&rows=10&fl=query&wt=json'
    url = 'http://localhost:8983/solr/popular_queries/select?q=query_edgengram:'+search_term+'&%20sort=product(popularity,reply_conversion)%20desc&rows=10&fl=query&wt=json'
    url2 = 'http://localhost:8983/solr/popular_queries/select?q=query_edgengram:'+search_term +'&%20sort='\
            'product(popularity,reply_conversion,pow(strdist(%22'+search_term+'%22,query,ngram),2))%20desc'\
            '&rows=10&fl=query&wt=json&fl=query,popularity,reply_conversion,research_quote,'\
            'first_cat_name,second_cat_name,second_cat_significance,first_cat_significance,'\
            'product(popularity,reply_conversion,pow(strdist(%22'+search_term+'%22,query,ngram),2)),'\
            'strdist(%22'+search_term+'%22,query,ngram),'\
            'pow(strdist(%22'+search_term+'%22,query,ngram),2),score&debugQuery=on'
    r = requests.get(url2).json()
    return [d.get('query', 'error')for d in r.get('response', {}).get('docs',{})], url2

In [611]:
import requests
def suggest(search_term, field):
    #url = 'http://localhost:8983/solr/solrpedia_instant/select?q=query_ngram:'+search_term+'&%20sort=popularity%20desc&rows=10&fl=query&wt=json'
    url = 'http://localhost:8983/solr/popular_queries/select?q='+field+':'+search_term+'&%20sort=product(popularity,reply_conversion)%20desc&rows=10&fl=query&wt=json'
    url2 = 'http://localhost:8983/solr/popular_queries/select?q='+field+':'+search_term +'&%20sort='\
            'product(popularity,reply_conversion,pow(strdist(%22'+search_term+'%22,query,ngram),2))%20desc'\
            '&rows=10&fl=query&wt=json&fl=query,popularity,reply_conversion,research_quote,'\
            'first_cat_name,second_cat_name,second_cat_significance,first_cat_significance,'\
            'product(popularity,reply_conversion,pow(strdist(%22'+search_term+'%22,query,ngram),2)),'\
            'strdist(%22'+search_term+'%22,query,ngram),'\
            'pow(strdist(%22'+search_term+'%22,query,ngram),2),score&debugQuery=on'
    r = requests.get(url2).json()
    return [d.get('query', 'error')for d in r.get('response', {}).get('docs',{})], url2

In [596]:
import requests
def suggest_sim(search_term):
    #url = 'http://localhost:8983/solr/solrpedia_instant/select?q=query_ngram:'+search_term+'&%20sort=popularity%20desc&rows=10&fl=query&wt=json'
    url2 = 'http://localhost:8983/solr/popular_queries/select?q=query_ngram:'+search_term +'&%20sort='\
            'product(popularity,pow(strdist(%22'+search_term+'%22,query,ngram),10))%20desc'\
            '&rows=10&fl=query&wt=json&fl=query,popularity,'\
            'product(popularity,pow(strdist(%22'+search_term+'%22,query,ngram),10)),'\
            'strdist(%22'+search_term+'%22,query,ngram),score&debugQuery=on'
    r = requests.get(url2).json()
    return [d.get('query', 'error')for d in r.get('response', {}).get('docs',{})], url2

In [627]:
s = suggest('onda', 'query_ngram')
for n in s[0]:
    print(n)
print(s[1])

honda
honda wave
honda biz
microondas
honda tornado
honda titan
honda civic
honda twister
honda fit
honda xr
http://localhost:8983/solr/popular_queries/select?q=query_ngram:onda&%20sort=product(popularity,reply_conversion,pow(strdist(%22onda%22,query,ngram),2))%20desc&rows=10&fl=query&wt=json&fl=query,popularity,reply_conversion,research_quote,first_cat_name,second_cat_name,second_cat_significance,first_cat_significance,product(popularity,reply_conversion,pow(strdist(%22onda%22,query,ngram),2)),strdist(%22onda%22,query,ngram),pow(strdist(%22onda%22,query,ngram),2),score&debugQuery=on


In [628]:
spell_check('qdida', 'olx')

http://localhost:8983/solr/olx/spell?df=query&spellcheck.q=qdida&spellcheck=true&spellcheck.collateParam.q.op=AND&wt=json


(u'queda',
 [{u'endOffset': 5,
   u'numFound': 8,
   u'origFreq': 0,
   u'startOffset': 0,
   u'suggestion': [{u'freq': 7, u'word': u'adida'},
    {u'freq': 161, u'word': u'queda'},
    {u'freq': 3, u'word': u'quita'},
    {u'freq': 1, u'word': u'qdaba'},
    {u'freq': 1, u'word': u'quiza'},
    {u'freq': 3, u'word': u'qeda'},
    {u'freq': 1, u'word': u'quda'},
    {u'freq': 1, u'word': u'quid'}],
   'token': u'qdida'}])

In [535]:
matches = []
for i, p in export[:1000].iterrows():
    for l in [3]:
        match = p['query'] in suggest(p['query'][:l])
        if not match:
            print(p['query'], suggest(p['query'][:l]))
        matches.append(match)

('fiat punto', [u'fiat', u'fiat 147', u'fiat uno', u'fiat 128', u'fiat 600', u'fiat duna', u'fiat palio', u'fiat siena', u'fiat 1', u'fiat 125'])
('motorhome', [u'motos', u'moto g', u'motos 110', u'moto 110', u'moto g3', u'moto g4', u'moto x', u'motor', u'motorola', u'moto g2'])
('volkswagen vento', [u'volkswagen gol', u'volvo', u'volkswagen bora', u'volkswagen saveiro', u'volkswagen gol trend', u'volkswagen fox', u'volkswagen amarok', u'volkswagen', u'volkswagen golf', u'volcom'])
('fiat uno scr', [u'fiat', u'fiat 147', u'fiat uno', u'fiat 128', u'fiat 600', u'fiat duna', u'fiat palio', u'fiat siena', u'fiat 1', u'fiat 125'])
('peugeot 308', [u'peugeot 206', u'peugeot', u'peugeot 504', u'peugeot 405', u'peugeot 306', u'peugeot 207', u'peugeot 307', u'peugeot partner', u'peugeot 505', u'peugeot 205'])
('chevrolet cruze', [u'chevy', u'chevrolet', u'chevrolet corsa', u'chevrolet s10', u'chery', u'cher', u'chevrolet c10', u'chevrolet 400', u'chevrolet astra', u'cheeky'])
('fiat fiorino', 

/Users/johannes.braun/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


('juego de ba\xc3\xb1o', [u'juegos ps4', u'juegos ps3', u'juegos de ps3', u'juegos', u'juegos de ps4', u'juegos de play 3', u'juego ps4', u'juego de living', u'juego ps3', u'juego de ba\xf1o'])
('camionetas en venta', [u'camas', u'camperas', u'camisa', u'camionetas', u'camiones', u'camion', u'camara', u'camiseta', u'cambio', u'camperon'])
('vw gol trend', [u'vw gol', u'vw fox', u'vw', u'vw bora', u'vw golf', u'vw polo', u'vw senda', u'vw up', u'vw amarok', u'vw saveiro'])
('camion volcador', [u'camas', u'camperas', u'camisa', u'camionetas', u'camiones', u'camion', u'camara', u'camiseta', u'cambio', u'camperon'])
('motos 150', [u'motos', u'moto g', u'motos 110', u'moto 110', u'moto g3', u'moto g4', u'moto x', u'motor', u'motorola', u'moto g2'])
('cama cucheta', [u'camas', u'camperas', u'camisa', u'camionetas', u'camiones', u'camion', u'camara', u'camiseta', u'cambio', u'camperon'])
('chevrolet luv', [u'chevy', u'chevrolet', u'chevrolet corsa', u'chevrolet s10', u'chery', u'cher', u'chev

UnicodeDecodeError: 'utf8' codec can't decode byte 0xc3 in position 66: invalid continuation byte

In [536]:
(sum(matches)+0.0)/len(matches)

0.8052910052910053

In [630]:
sum(matches)

761

In [629]:
len(matches)

945

In [567]:
def transform_tolist(sc):
    d = []
    while len(sc) >= 2:
        sc[1]['token'] = sc[0]
        d.append(sc[1])
        sc = sc[2:]
    return d 

In [568]:
def spell_check(search_term, index):
    #url = 'http://localhost:8983/solr/letgoar/select?q='+search_term+'&df=suggest&wt=json'
    url = 'http://localhost:8983/solr/'+index+'/spell?df=query&spellcheck.q='+search_term+'&spellcheck=true&spellcheck.collateParam.q.op=AND&wt=json'
    print(url)
    r = requests.get(url).json()
    suggestions = r.get('spellcheck', {}).get('suggestions', [])
    suggestions = transform_tolist(suggestions)
    checked = search_term
    for s in suggestions:
        t = s['token']
        freq = s['origFreq']
        for ss in s['suggestion']:
            if freq < ss['freq']:
                t = ss['word']
                freq = ss['freq']
        checked = checked.replace(s['token'], t)
    return checked , suggestions

In [571]:
suggest('adida')

([u'adidas',
  u'zapatillas adidas',
  u'campera adidas',
  u'adidas superstar',
  u'buzo adidas',
  u'conjunto adidas',
  u'camperas adidas',
  u'ropa adidas',
  u'botines adidas',
  u'pantalon adidas'],
 'http://localhost:8983/solr/popular_queries/select?q=query_ngram:adida&%20sort=product(popularity,reply_conversion,pow(strdist(%22adida%22,query,ngram),2))%20desc&rows=10&fl=query&wt=json&fl=query,popularity,reply_conversion,research_quote,first_cat_name,second_cat_name,second_cat_significance,first_cat_significance,product(popularity,reply_conversion,pow(strdist(%22adida%22,query,ngram),2)),strdist(%22adida%22,query,ngram),pow(strdist(%22adida%22,query,ngram),2),score&debugQuery=on')

In [569]:
spell_check("adida", 'popular_queries')

http://localhost:8983/solr/popular_queries/spell?df=query&spellcheck.q=adida&spellcheck=true&spellcheck.collateParam.q.op=AND&wt=json


(u'adidas',
 [{u'endOffset': 5,
   u'numFound': 10,
   u'origFreq': 0,
   u'startOffset': 0,
   u'suggestion': [{u'freq': 218, u'word': u'adidas'},
    {u'freq': 1, u'word': u'adidaa'},
    {u'freq': 1, u'word': u'adidad'},
    {u'freq': 5, u'word': u'adda'},
    {u'freq': 3, u'word': u'akira'},
    {u'freq': 3, u'word': u'akita'},
    {u'freq': 2, u'word': u'addidas'},
    {u'freq': 2, u'word': u'adiddas'},
    {u'freq': 1, u'word': u'adela'},
    {u'freq': 1, u'word': u'awada'}],
   'token': u'adida'}])

In [570]:
spell_check("adida", 'olx')

http://localhost:8983/solr/olx/spell?df=query&spellcheck.q=adida&spellcheck=true&spellcheck.collateParam.q.op=AND&wt=json


(u'adidas',
 [{u'endOffset': 5,
   u'numFound': 5,
   u'origFreq': 7,
   u'startOffset': 0,
   u'suggestion': [{u'freq': 586, u'word': u'adidas'},
    {u'freq': 3, u'word': u'adidad'},
    {u'freq': 1, u'word': u'aadida'},
    {u'freq': 1, u'word': u'addida'},
    {u'freq': 1, u'word': u'adita'}],
   'token': u'adida'}])